# 🙌 Praktická ukázka

V tomto notebooku si vizualizace ukážeme na reálném modelu. Využijeme k tomu 🏘 [California Housing Dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) a 🌳 `DecisionTreeRegressor`.

Cílem této ukázky **není** vytvořit skvělý ML model, zajímají nás jenom vizualizace.

Začneme importem potřebných balíčků a definicí barev, které budeme dále využívat.

In [ ]:
# plotting
import matplotlib.pyplot as plt
import seaborn as sns
from dtreeviz.trees import dtreeviz

# model
from sklearn.tree import DecisionTreeRegressor

# data generation and preparation
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd

# hyperparameter tuning
import sklearn.metrics as metrics
from sklearn.model_selection import ParameterGrid

# interactive elements
from ipywidgets import interact

In [ ]:
# colors
blue = '#8592dc'
violet = '#9047A0'
red = '#d14081'
grey = '#8E8DB4'
white = '#ffffff'
black='#222222'
green='#42aa78'

## 📚 Dataset
V datasetu máme 8 numerických příznaků a jednu vysvětlovanou proměnnou MedHouseVal.

In [ ]:
california = datasets.fetch_california_housing(as_frame=True)
X = california.data
y = california.target

In [ ]:
california.frame #shows data as DataFrame, when as_frame=True

Data si rozdělíme na trénovací, validační a testovací. ☝️ Validační data použijeme k ladění hyperparametrů a testovací k vyhodnocení finální úspěšnosti modelu (znáte z BI-ML1).

In [ ]:
def split_data(X, y, rd_seed=1234):
    # split data to train (75%) and test (25%)
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=rd_seed) 
    # split train data from previous step to train (75%) and validation (25%)
    Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=rd_seed) 
    return Xtrain, Xtest, Xval, ytrain, ytest, yval

Xtrain, Xtest, Xval, ytrain, ytest, yval = split_data(X,y)

## 🛠 Ladění hyperparametrů

Budeme ladit tři hyperparametry:
* 👇🏽 max_depth
* 🍃 min_samples_leaf
* 🙅🏽‍♀️ min_samples_split

Na vygenerování všech kombinací těchto hyperparametrů využijeme [`ParameterGrid`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html) (tj. mřížka parametrů s diskrétním počtem hodnot pro každý z nich).

In [ ]:
params_dict = {
    'max_depth': range(2,20,2),
    'min_samples_leaf': [1, 3, 5, 7],
    'min_samples_split': [2, 3, 5, 7]
}

params_grid = ParameterGrid(params_dict)

Nyní vytvoříme a následně zavoláme funkci `tune`, která na základě trénovacích a validačních dat a kombinací hyperparametrů, které chceme zkusit, natrénuje různé modely a změří jejich trénovací a validační chybu. Jako metriku jsme zvolili MSE (mean square error). Funkce vrátí dvě pole, `train_mse` (trénovací MSE) a `val_mse` (validační MSE).

Hodnota `train_mse[i]` odpovídá trénovací chybě modelu s hyperparametry `params_comb[i]`.

In [ ]:
def tune(Xtrain, ytrain, Xval, yval, params_comb):
    train_mse = []
    val_mse = []

    for params in params_comb:
        dt = DecisionTreeRegressor(**params)
        dt.fit(Xtrain, ytrain)
        val_mse.append(metrics.mean_squared_error(yval, dt.predict(Xval)))
        train_mse.append(metrics.mean_squared_error(ytrain, dt.predict(Xtrain)))
        
    return train_mse, val_mse

train_mse, val_mse = tune(Xtrain, ytrain, Xval, yval, params_grid)

A teď je čas na vizualizaci. Už jsme si ukazovali, jak vykreslit vývoj nějaké metriky u jednoho hyperparametru. Nyní jich máme víc. Naštěstí můžeme použít úplně stejný graf. Osa x nyní zobrazuje index v poli všech kombinací hyperparametrů.

Pro lepší přehlednost lze přidat vertikální čáry, které ukazují, kdy došlo ke změně hodnoty nějakého hyperparametru.

In [ ]:
def plot_mse(train_mse, val_mse):
    fig, ax = plt.subplots(figsize=(16,8))

    for x in range(16,144,16):
        plt.axvline(x=x, c=white, linewidth=3, label='max_depth change')
    for x in range(4,144,4):
        plt.axvline(x=x, c=white, label='min_samples_leaf change')

    ax.plot(train_mse, 'o-', color=blue, label='train mse')
    ax.plot(val_mse,'o-', color=violet, label='validation mse')

    # legend with duplicated labels removed, for more info see:
    # https://stackoverflow.com/questions/13588920/stop-matplotlib-repeating-labels-in-legend
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), facecolor=grey, framealpha=0.4)
    
    # styling  
    ax.set_xlim(left=-1, right=144)
    ax.set(facecolor = "#eaeaf2")
    for key, spine in ax.spines.items():
        spine.set_visible(False)  
    
    # labels
    ax.set_xlabel('index in parameter array')
    ax.set_ylabel('mean squared error')
    ax.set_title('Train and test MSE for every hyperparameter combination')
    
plot_mse(train_mse, val_mse)

Vypadá to tak, že u malých hloubek vůbec nezáleželo na hodnotách ostatních hyperparametrů, protože se chybovost příliš neměnila.

V pravé části grafu, kde už je hloubka větší, je vidět i vliv hyperparametru 🍃 `min_samples_leaf`.
S narůstající hodnotou hyperparametru na validačních datech MSE klesá (s rostoucí hloubkou MSE vzroste) a na trénovacích MSE roste (s rostoucí hloubkou MSE klesá). To svědčí o tom, že stromy, kterým jsme dovolili vytvářet listy s malým počtem pozorování, byly přeučeny.

Nyní se můžeme posunout dál a **vybrat nejlepší hyperparametry**. To jsou ty, které byly použity k natrénování modelu s nejnižší **validační MSE**. Na těchto hyperparametrech znovu natrénujeme model.

In [ ]:
best_params = params_grid[np.argmin(val_mse)]
print('Nejlepší hyperparametry pro validační data jsou:', best_params)

dt = DecisionTreeRegressor(**best_params)
dt.fit(Xtrain, ytrain)

## ⚖️ Evaluace
Začneme tím, že vytvoříme predikce pro trénovací, validační a testovací data.

In [ ]:
ytrain_pred = dt.predict(Xtrain)
yval_pred = dt.predict(Xval)
ytest_pred = dt.predict(Xtest)

Pro všechny tři množiny dat nyní máme reálné hodnoty i predikce. Můžeme tak vytvořit `DataFrame`, který obsahuje reálnou a predikovanou hodnotu, jejich rozdíl (chybu) a informaci o tom, do které množiny data patří. 🕵🏻‍♂️

In [ ]:
def create_error_df(yreal, ypred, subset):
    df = pd.DataFrame()
    df['real'] = yreal
    df['predicted'] = ypred
    df['error'] = yreal - ypred
    df['set'] = subset
    return df
    
df = pd.concat([
    create_error_df(ytrain, ytrain_pred, 'train'),
    create_error_df(yval, yval_pred, 'validation'),
    create_error_df(ytest, ytest_pred, 'test'),
])

display(df.head())
df.describe()

Závažnost chyby budeme počítat s ohledem na rozsah vysvětlované proměnné. Každou chybu vydělíme rozdílem maximální a minimální hodnoty na trénovací množině.

In [ ]:
df['normalized_error'] = df.error / (ytrain.max() - ytrain.min())
df['normalized_error'].describe()

In [ ]:
def compute_severity(row):
    if abs(row.normalized_error) < 0.1:
        return 'low'
    if abs(row.normalized_error) < 0.3:
        return 'medium'
    return 'high'

df['severity'] = df.apply(compute_severity, axis=1)

V notebooku `evaluations.ipynb` jsme si ukázali, jak vytvořit graf reálné a predikované hodnoty. Nyní do tohoto grafu přidáme také informaci o závažnosti chyby, kterou budeme zobrazovat pomocí barvy. Abychom mohli porovnat trénovací, validační a testovací chybovost, pro každou z těchto množin vytvoříme samostatný graf. Pomocí interaktivních prvků umožníme výběr množiny, pro kterou se graf zobrazí.

In [ ]:
def plot_real_pred_interactive(dataframe):
    
    def plot_real_pred(subset):
        df = dataframe[dataframe.set == subset]
        with plt.style.context('seaborn-darkgrid'):
            low = df[df.severity == 'low']
            medium = df[df.severity == 'medium']
            high = df[df.severity == 'high']

            fig, ax = plt.subplots(figsize=(16,8))
            ax.plot([0,5.5],[0,5.5], color=black, label='x = y')
            ax.scatter(low.real, low.predicted, color=green, alpha=0.15, label='Low severity')
            ax.scatter(medium.real, medium.predicted, color=blue, alpha=0.15, label='Medium severity')
            ax.scatter(high.real, high.predicted, color=red, alpha=0.15, label='High severity')

            ax.set_xlabel('Real value')
            ax.set_ylabel('Predicted value')
            ax.legend()
            ax.set_title('Error visualization for {} batch'.format(subset))

    interact(plot_real_pred, subset=['train', 'validation', 'test'])
    
plot_real_pred_interactive(df)

Na první pohled je vidět, že na validačních a testovacích datech máme v porovnání s trénovacími více velkých chyb (vizualizovaných červenou barvou).

Pojďme se podívat na distribuci chyby. Vykreslíme si violin plot a normalizovaný histogram.

In [ ]:
# seaborn default theme with dark grid
sns.set_theme()

fig, ax = plt.subplots(figsize=(16,8))
sns.violinplot(ax=ax, data=df, x='set', y='error', palette='Set2')

ax.set_ylabel('Error')
ax.set_xlabel('Set')
_ = ax.set_title('Error across sets')

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
sns.histplot(
    df, binwidth=0.05, x='error', hue='set', palette='Set2', 
    element='step', ax=ax, stat='probability', common_norm=False,
)

# stat='probability' - normalize such that bar heights sum to 1

ax.set_xlabel('Error')
ax.set_ylabel('Normalized count')
_ = ax.set_title('Normalized histogram of error across sets')

Z obou grafů vidíme, že na trénovacích datech model funguje nejlépe (většina chyb je velmi malých a větší chyby nastávají méně často). Na validační a testovací množině dělal model přibližně stejné chyby. Na obou množinách jsou větší chyby frekventovanější než na trénovací množině.

Jako poslední vizualizaci vykreslíme výsledný model pomocí balíčku `dtreeviz` 🌳. Jelikož má model velkou hloubku, je rozumné nezobrazovat bodové grafy v každém vrcholu, ale pouze v listech. Toho dosáhneme pomocí parametru `fancy=False`. I tak si na vizualizaci budeme muset chvíli počkat 🕒.

In [ ]:
viz = dtreeviz(
    dt, Xtrain, ytrain,
    target_name='price',
    feature_names=california.feature_names,
    class_names=list(california.target_names),
    scale=1.5,
    fancy=False
)

viz

## 🎉 Pro dnešek máme hotovo! 🎉